# Setup

In [ ]:
import importlib

if importlib.util.find_spec('ipywidgets') is None:
    !pip install ipywidgets
if importlib.util.find_spec('yt-dlp') is None:
    !pip install yt-dlp

# OSS
from concurrent.futures import ProcessPoolExecutor
from IPython.display import display
from threading import Thread
from typing import Dict
from typing import List
from typing import TYPE_CHECKING
from typing import TypedDict
import getpass
import ipywidgets as widgets
import json
import os

def is_running_on_colab():
    return 'COLAB_GPU' in os.environ

# 独自モジュール
if TYPE_CHECKING or not is_running_on_colab():
    from my_package.format_data import replace_watch_comment
    from my_package.get_chat import get_chat
    from my_package.gpt_handler import チャットGPTハンドラークラス
else:
    from commentSupporter.src.format_data import replace_watch_comment
    from commentSupporter.src.get_chat import get_chat
    from commentSupporter.src.gpt_handler import チャットGPTハンドラークラス


In [ ]:
youtube_live_url = input("YoutubeのURLを入力してください: ")
api_key = getpass.getpass("chatGPTを利用する方は、API keyを自己責任で入力してください。: ")
split_url = youtube_live_url.split('https://www.youtube.com/watch?v=')
assert split_url[0] == ''
live_id:str = split_url[1]

# Promptの設定

In [ ]:
チャットGPTハンドラー = チャットGPTハンドラークラス(api_key, "あなたはコメント分析の専門家です")
チャットGPTハンドラー.ユーザーメッセージの追加(
    """
        次の一連のコメントから、コメント主の特徴を一言で分析してください。
        「昨日はラーメン食べた」
        「今日はパスタ食べた」
        「明日はうどん食う」
    """)
チャットGPTハンドラー.chatGPT返信の追加("麺好き")
チャットGPTハンドラー.ユーザーメッセージの追加(
    """
        素晴らしい。
        次の一連のコメントから、コメント主の特徴を一言で分析してください。
        「昨日カレー食べた」
        「今日はカレーグラタン食べた」
        「明日はカレーうどん食う」
    """)
チャットGPTハンドラー.chatGPT返信の追加("カレー好き")
チャットGPTハンドラー.ユーザーメッセージの追加(
    """
        素晴らしい。次の一連のコメントから、コメント主の特徴を一言で分析してください。
    """)

# 中略

In [ ]:
class chat_transaction(TypedDict):
    user_id: str
    display_name: str
    chat: str
    date: str

class CommentData(TypedDict):
    displayName: str
    date: str
    comment: str
    userId: str

list_1: list[chat_transaction]
select_1: widgets.SelectMultiple
list_2:  list[CommentData]
select_2: widgets.SelectMultiple
title_2  : widgets.Label
footer_2  : widgets.Label
live_id:str
チャットGPTハンドラー: チャットGPTハンドラークラス


def load_chat(live_id: str) -> list[chat_transaction]:
    """指定したパスのJSONファイルを読み込みます。ファイルが存在しない場合は新しいリストを返します。

    Args:
        file_path (str): JSONファイルへのパス。

    Returns:
        List[Any]: JSONから読み込んだデータ、または新しいリスト。
    """
    if os.path.exists(f"{live_id}_chat.json"):
        with open(f"{live_id}_chat.json", 'r', encoding='utf-8') as f:
            data:list[chat_transaction] = json.load(f)
    else:
        data:list[chat_transaction] = []  # ファイルが存在しない場合は新しいリストを作成
    # print(data)
    return data

def load_each_chat(live_id: str) -> Dict[str, List[CommentData]]:
    """指定したパスのJSONファイルを読み込みます。ファイルが存在しない場合は新しいリストを返します。

    Args:
        file_path (str): JSONファイルへのパス。

    Returns:
        List[Any]: JSONから読み込んだデータ、または新しいリスト。
    """
    if os.path.exists(f"{live_id}_each_chat.json"):
        with open(f"{live_id}_each_chat.json", 'r', encoding='utf-8') as f:
            data:Dict[str, List[CommentData]] = json.load(f)
    else:
        data:Dict[str, List[CommentData]] = {}  # ファイルが存在しない場合は新しいリストを作成
    return data

# クリックイベントのハンドラー
def update(button:widgets.Button):
    global list_1
    global select_1 
    global live_id
    try:
        list_1= load_chat(live_id)[:-150]
        scroll_end_message = [f"もうすぐスクロール端です：{i}" for i in range(5)] if 30<len(list_1) else []
        list_all = scroll_end_message + [f"{el['display_name']}：{el['chat']}" for el in list_1] + scroll_end_message[::-1]
        select_1.options = list_all
    except Exception as error:
        print('----------')
        print('error')
        print(error)


def analyze(change):
    global list_1
    global list_2 
    global select_1 
    global select_2 
    global title_2 
    global footer_2
    global チャットGPTハンドラー
    if change['new']:
        print('aaa')
        name:str =change['new'][0].split('：')[0]
        assert isinstance(name, str)
        print(name)
        print(type(name))
        if name in load_each_chat(live_id) :
            # "addList"を10回list_1に追加
            footer_2.value= "chatGPT：分析中..."
            list_2= load_each_chat(live_id)[name]
            scroll_end_message = [f"もうすぐスクロール端です：{i}" for i in range(5)] if 30<len(list_2) else []
            list_all = scroll_end_message + [f"{el['date']}：{el['comment']}" for el in list_2] + scroll_end_message[::-1]
            # select_1のoptionsを更新
            select_2.options = list_all
            title_2.value =  "コメント主："+name
            footer_2.value= "chatGPT：" + チャットGPTハンドラー.チャットGPTへ問いかけ([f"{el['comment']}" for el in list_2][::-1])

def set_ui():
    # リストの内容
    global list_1
    global list_2 
    global select_1 
    global select_2 
    global title_2 
    global footer_2 
    list_1 = ['ロード中です：ロード中です']
    list_2 = []

    # スクロール端メッセージの追加
    scroll_end_message = [f"1{i}" for i in range(5)] if 30< len(list_2) else [] 
    list_1_all = scroll_end_message + list_1 + scroll_end_message[::-1]
    list_2_all = scroll_end_message + list_2 + scroll_end_message[::-1]

    # SelectMultiple widgetsの作成
    select_1 = widgets.SelectMultiple(
        options=list_1_all,
        description='',  # リストタイトルを非表示
        layout=widgets.Layout(width='auto', height='580px', overflow_y='auto')  # スクロールを追加
    )

    select_2 = widgets.SelectMultiple(
        options=list_2_all,
        layout=widgets.Layout(width='auto', height='500px', overflow_y='auto')  # スクロールを追加
    )

    # イベントハンドラーの追加
    select_1.observe(analyze, 'value')

    # タイトルウィジェットの作成
    title_2 = widgets.Label(
        value='"コメント主："', 
        layout=widgets.Layout(width='auto')
    )

    # フッターウィジェットの作成
    footer_2 = widgets.Label(
        value="chatGPT：", 
        layout=widgets.Layout(width='auto')
    )

    # 右側のカラムを作成（タイトル、リスト、フッターを含む）
    right_column = widgets.VBox([title_2, select_2, footer_2], layout=widgets.Layout(width='50%', overflow='auto'))
    
    # ボタンウィジェットを作成
    update_button = widgets.Button(description="Update")
    update_button.on_click(update) 

    # フッターウィジェットの作成
    footer_1 = widgets.HBox([widgets.Label(value=''), update_button], 
                            layout=widgets.Layout(width='auto'))

    # 左側のカラムを作成（リストとフッターを含む）
    left_column = widgets.VBox([select_1, footer_1], layout=widgets.Layout(width='50%', overflow='auto'))

    # 2カラムのレイアウトを作成
    two_columns = widgets.HBox([left_column, right_column], layout=widgets.Layout(width='100%', overflow='auto'))

    display(two_columns)


In [ ]:
def main(live_id:str):
    set_ui()
    # プロセスを作成
    executor = ProcessPoolExecutor(max_workers=2)  # ワーカーの数を2に戻します

    # プロセスを開始（別スレッドで実行）
    t1 = Thread(target=lambda: executor.submit(replace_watch_comment, live_id, チャットGPTハンドラー))
    t1.start()

    t2 = Thread(target=lambda: executor.submit(get_chat, live_id))
    t2.start()

# 下記に、チャットとその分析結果が表示されます
## 初回のチャットが分析されるまで、数十秒かかる場合があります

In [ ]:

# # プロセスを作成
# executor = ProcessPoolExecutor(max_workers=2)  # ワーカーの数を2に戻します


# # プロセスを開始（別スレッドで実行）
# t1 = Thread(target=lambda: executor.submit(replace_watch_comment, live_id, チャットGPTハンドラー))
# t1.start()

# t2 = Thread(target=lambda: executor.submit(get_chat, live_id))
# t2.start()

# set_ui()
main(live_id)